In [1]:
#####
# 1.最新四半期決算の営業利益率は低い
# 2.最新四半期決算の売上高成長率が10%以上
# 3.最新四半期決算の対売上高営業利益成長率が20%以上
# 4.次期決算を、以下の方法で予測した場合において、営業利益率が前年同期比2倍以上になる
# --> 最新四半期決算の売上高成長率と対売上高営業利益成長率を、次期前年四半期決算に適用して次期決算を予想する

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [4]:
KPL = KessanPl()
KPL.with_columns_lastyear_settlement()
df1 = KPL.df.filter(pl.col("settlement_type")=="四")
df1 = df1.with_columns([
    pl.col("settlement_date").alias("key")
])
df1

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater,ly_settlement_date,ly_announcement_date,ly_sales,ly_operating_income,ly_ordinary_profit,ly_final_profit,key
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64,date,date,i64,i64,i64,i64,date
1301,2019-09-30,"""四""",2019-11-05,64240,-35,113,4,0.4,-0.1,2,2018-09-30,2018-11-05,61245,507,595,269,2019-09-30
1301,2019-12-31,"""四""",2020-02-07,78552,1881,1851,1029,95.1,2.4,3,2018-12-31,2019-02-08,78581,2208,2591,1677,2019-12-31
1301,2020-03-31,"""四""",2020-05-12,57083,302,629,298,27.6,0.5,4,2019-03-31,2019-05-13,58368,551,511,413,2020-03-31
1301,2020-06-30,"""四""",2020-08-07,54320,633,697,565,52.6,1.2,1,2019-06-30,2019-08-02,62644,770,1015,706,2020-06-30
1301,2020-09-30,"""四""",2020-11-06,62541,746,808,1012,94.2,1.2,2,2019-09-30,2019-11-05,64240,-35,113,4,2020-09-30
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9997,2023-09-30,"""四""",2023-10-31,44263,1939,2466,1878,19.4,4.4,2,2022-09-30,2022-10-31,45340,1936,2219,1467,2023-09-30
9997,2023-12-31,"""四""",2024-01-31,60173,2630,2605,1818,18.8,4.4,3,2022-12-31,2023-01-31,61675,4185,4165,2533,2023-12-31
9997,2024-03-31,"""四""",2024-05-13,52020,4062,4901,758,7.8,7.8,4,2023-03-31,2023-05-12,51922,3343,3623,2035,2024-03-31


In [5]:
# para
valuation_date = date(2024, 8, 1)

KPL = KessanPl()
df2 = KPL.get_settlement_forcast_by_diff_growth_rate(valuation_date)
df2 = df2.filter(pl.col("settlement_type")=="四")
df2 = df2.with_columns([
    pl.col("nxt_settlement_date").alias("key")
])
df2

code,announcement_date,settlement_date,settlement_type,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,col,nxt_settlement_date,key
i64,date,date,str,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,date,date,date
1301,2024-05-10,2024-03-31,"""四""",59990,1857,1863,951,87.9,3.1,4,61250,3823,3979,2143,2024-06-19,2024-06-30,2024-06-30
1332,2024-05-14,2024-03-31,"""四""",205894,3308,4183,3557,11.4,1.6,4,223395,5863,6931,4730,2024-06-19,2024-06-30,2024-06-30
1333,2024-05-07,2024-03-31,"""四""",243328,1176,878,-1581,-31.4,0.5,4,247221,-237,-858,-3216,2024-06-19,2024-06-30,2024-06-30
1375,2024-05-09,2024-03-31,"""四""",11807,-803,-1015,-774,-19.4,-6.8,4,14298,-903,-1247,-1018,2024-06-19,2024-06-30,2024-06-30
1376,2024-07-10,2024-05-31,"""四""",23224,1150,1134,918,80.6,5.0,4,23665,1084,1042,911,2024-08-19,2024-08-31,2024-08-31
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9993,2024-07-11,2024-05-31,"""四""",24969,-408,-223,-186,-17.3,-1.6,1,25418,-891,-539,-1045,2024-08-19,2024-08-31,2024-08-31
9994,2024-05-15,2024-03-31,"""四""",36543,1082,1070,581,53.6,3.0,4,37676,1816,1779,1041,2024-06-19,2024-06-30,2024-06-30
9995,2024-05-07,2024-03-31,"""四""",18708,63,249,433,15.2,0.3,4,20149,79,452,842,2024-06-19,2024-06-30,2024-06-30


In [13]:
df = df2.join(df1, on=["code", "key"], how="left")
df = df.select([
    "code",	
    "settlement_date",
    "announcement_date",
    "settlement_type",
    'sales',
    'operating_income',
    'ordinary_profit',
    'final_profit',
    "quater",
    "ly_settlement_date",
    "ly_sales",
    "ly_operating_income",
    "ly_ordinary_profit",
    "ly_final_profit",
    "nxt_settlement_date",
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",	
    "fcst_dgr_final_profit"	
])
df

code,settlement_date,announcement_date,settlement_type,sales,operating_income,ordinary_profit,final_profit,quater,ly_settlement_date,ly_sales,ly_operating_income,ly_ordinary_profit,ly_final_profit,nxt_settlement_date,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit
i64,date,date,str,i64,i64,i64,i64,i64,date,i64,i64,i64,i64,date,i64,i64,i64,i64
1301,2024-03-31,2024-05-10,"""四""",59990,1857,1863,951,4,2023-06-30,62367,2421,2548,1827,2024-06-30,61250,3823,3979,2143
1332,2024-03-31,2024-05-14,"""四""",205894,3308,4183,3557,4,2023-06-30,199844,9720,9718,5981,2024-06-30,223395,5863,6931,4730
1333,2024-03-31,2024-05-07,"""四""",243328,1176,878,-1581,4,2023-06-30,250922,7685,11213,6652,2024-06-30,247221,-237,-858,-3216
1375,2024-03-31,2024-05-09,"""四""",11807,-803,-1015,-774,4,2023-06-30,8274,-315,-413,-283,2024-06-30,14298,-903,-1247,-1018
1376,2024-05-31,2024-07-10,"""四""",23224,1150,1134,918,4,2023-08-31,13727,258,309,208,2024-08-31,23665,1084,1042,911
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9993,2024-05-31,2024-07-11,"""四""",24969,-408,-223,-186,1,2023-08-31,26129,454,473,348,2024-08-31,25418,-891,-539,-1045
9994,2024-03-31,2024-05-15,"""四""",36543,1082,1070,581,4,2023-06-30,38910,1322,1362,797,2024-06-30,37676,1816,1779,1041
9995,2024-03-31,2024-05-07,"""四""",18708,63,249,433,4,null,null,null,null,null,2024-06-30,20149,79,452,842
